# Programming Assignment 5: Entity Resolution

In this assignment you are give a  dataset that contains product data from Walmart and Amazon. The dataset contains two tables the left table (denoted by *ltable*), and the right table (denoted by *rtable*). Further, we are providing a dataset (*test.csv*) that contains a list of true product matches and no-matches for evaluation.

Table ltable has the following schema: ltable_id, title, category, brand, modelno, price

Table rtable has the following schema: rtable_id, title, category, brand, modelno, price

The testing data file has the following schema: ltable_id, rtable_id, label, id 

In the testing data file the label is 0 if the two products do not match and 1 if they match. 

*Helpful hint: to read the two tables you will need to use the encoding latin-1*

For this task you will need to solve the problem of entity matching over the provided records of products and then report the performance of your solution when evaluated on the testing dataset. 

You are expected to submit a notebook that implements the following:

1. Implement an entity matching solution between the two tables, using the Record linkage toolkit we introduced in class.
  + You can use any index you need as long as your notebook can be executed within a reasonable time frame. **Entity matching code that takes more than 5 mins to run will be getting zero points**. You can measure the time it takes for a cell to execute by using the command <code> %%time* </code> as the first line of your cell. 
  + *Hint* : A full index will take a long time to execute. You should be mindful of the number of product comparisons your code will execute. A blocked index on the other hand might miss some identical products if it looking for matches on the "wrong" set of attributes, if it is looking for exact strings vs similar strings, if the matching threshold is set to be too high, etc. You could use any of the indexing techniques we discussed in class or any others that you see fit based on the documentation of the [Record linkage toolkit](https://recordlinkage.readthedocs.io/en/latest/about.html) 
  
2. Report the final prediction for your matches in an output csv file that has the schema <ltable_id, rtable_id, prediction>. First line should have be this: ltable_id,rtable_id,prediction. Name your output file output.csv. 
  + Prediction should have the value 1 if the two products match and 0 otherwise. Include the output file in your submittion. 
  + How you decide the final prediction depends on the approach you used (and your intuition). For instance, if you are using 3 strings (title, category, brand) for your matching then you needs to decide if two products match if they agree on all or some of these 3 strings. 
  
3. Report the false positives and false negatives by comparing your matches with the ground truth (which is in the testing dataset test.csv we provided). You should also report the precision and recall of your solution and the F1 measure. 
  + We will also test the F1 measure of your matches. **The top-5 F1 measures in class will get 10 extra credit points**. This assignment is a competition!  Our solution got a 0.76 F1 measure but you could possibly do even better.  
  + Note: some of the product matches  (pairs of <ltable_id, rtable_id) > that appear in the test file we provided (the ground truth), might not appear in your  output of your and vice versa. We are asking you to evaluate the F1 measure in the intersection of those two sets. 
  
4. Describe in a markdown cell in your notebook the process you used for your solution. Why did you choose the specific index, why the specific attributes (if any) for your blocked index, why did you use or not the SortedNeighborhood algorithm for your index. We want to understand the process you used and we expect it to be  a bit more sophisticated  than  a random pick of parameters. In other words - defend your notebook! 

Good luck! 

# Part 1
1. Implement an entity matching solution between the two tables, using the Record linkage toolkit we introduced in class.


In [1]:
import pandas as pd
from pathlib import Path
import recordlinkage

ltable = pd.read_csv('ltable.csv', index_col= "ltable_id", encoding='latin-1')
rtable = pd.read_csv('rtable.csv', index_col= "rtable_id", encoding='latin-1')

In [2]:
#clean the data from the csv's
ltable['title'] = ltable['title'].astype(str)
rtable['title'] = rtable['title'].astype(str)
ltable['category'] = ltable['category'].astype(str)
rtable['category'] = rtable['category'].astype(str)
ltable['brand'] = ltable['brand'].astype(str)
rtable['brand'] = rtable['brand'].astype(str)
ltable['modelno'] = ltable['modelno'].astype(str)
rtable['modelno'] = rtable['modelno'].astype(str)
ltable['price'] = ltable['price'].astype(str)
rtable['price'] = rtable['price'].astype(str)
print(ltable.dtypes)
print()
print(rtable.dtypes)

title       object
category    object
brand       object
modelno     object
price       object
dtype: object

title       object
category    object
brand       object
modelno     object
price       object
dtype: object


In [3]:
ltable.head()

,title,category,brand,modelno,price
ltable_id,,,,,
0,draper infrared remote transmitter,electronics - general,draper,121066,58.45
1,epson 1500 hours 200w uhe projector lamp elplp12,monitors,epson,elplp12,438.84
2,comprehensive two-piece 75 precision bnc jack ...,tv accessories,comprehensive,bj-2c7559,59.25
3,d-link dcs-1100 network camera,garden - general,d-link,dcs-1100,99.82
4,startech.com rkpw247015 24 outlet power strip,electronics - general,startech,rkpw247015,59.0


In [4]:
rtable.head()

,title,category,brand,modelno,price
rtable_id,,,,,
0,koss eq50 3-band stereo equalizer,headphone accessories,koss,152132,12.65
1,kodak black ink cartridge 10b 1163641,inkjet printer ink,kodak,1163641,10.28
2,kingston 128mx64 pc2700 compaq evo d320 ktc-d3...,computers accessories,kingston,ktc-d320 / 1g,33.75
3,kinamax ms-ues2 mini high precision usb 3-butt...,mice,kinamax,ms-ues2,6.99
4,kensington k72349us wireless mouse for netbooks,mice,kensington,k72349us,24.0


In [5]:
%%time
# Build the indexer
indexer = recordlinkage.Index()

#sortedneighbor accounts for some variations in the spelling of the brand name
indexer.sortedneighbourhood(left_on='brand', right_on='brand')
candidates = indexer.index(ltable, rtable)

compare = recordlinkage.Compare()
#only want exact matches for model numbers as some may be close to each other and referring to different entities
compare.exact('modelno', 'modelno', label='modelno')
#there can be some variation in the categories and how they are recorded, so the threshold reflects that
#used default of levenshtein to measure the number of changes
compare.string('category',
               'category',
               threshold=0.51,
               label='category')
#use jarowinkler because it measures the characters in common and
#it has higher F_1 score 
compare.string('title',
               'title',
               method='jarowinkler',
               threshold=0.67,
               label='title')
features = compare.compute(candidates, ltable, rtable)

Wall time: 9.13 s


In [6]:
features
features.sum(axis = 1).value_counts().sort_index(ascending = False)

3.0        65
2.0      5922
1.0     75880
0.0    233104
dtype: int64

In [7]:
#only send score of 1, 2, or 3 to the csv
potential_matches = features[features.sum(axis=1) >= 1].reset_index()
potential_matches['prediction'] = potential_matches.loc[:, 'modelno' : 'title'].sum(axis = 1)

ltable['Product Information_L'] = ltable[[
    'title', 'category', 'brand', 'modelno', 'price'
]]. apply(lambda x: '_'.join(x), axis = 1)

rtable['Product Information_R'] = rtable[[
    'title', 'category', 'brand', 'modelno', 'price'
]]. apply(lambda x: '_'.join(x), axis = 1)


ltable_lookup = ltable[['Product Information_L']].reset_index()
rtable_lookup = rtable[['Product Information_R']].reset_index()

potential_matches

,ltable_id,rtable_id,modelno,category,title,prediction
0,11,20440,0,0.0,1.0,1.0
1,12,20440,0,0.0,1.0,1.0
2,45,12911,0,0.0,1.0,1.0
3,45,20440,0,0.0,1.0,1.0
4,77,12911,0,0.0,1.0,1.0
...,...,...,...,...,...,...
81862,2418,11689,0,0.0,1.0,1.0
81863,2486,4637,1,0.0,0.0,1.0
81864,2486,17731,1,0.0,0.0,1.0
81865,2486,17732,1,0.0,0.0,1.0


# Part 2
Report the final prediction for your matches in an output csv file that has the schema <ltable_id, rtable_id, prediction>. First line should have be this: ltable_id,rtable_id,prediction. Name your output file output.csv.

In [8]:
product_merge = potential_matches.merge(ltable_lookup, how = 'left')
final_merge = product_merge.merge(rtable_lookup, how = 'left')
print(final_merge.dtypes)

final_merge.loc[final_merge['prediction'] >= 2, 'prediction'] = 1
final_merge.loc[final_merge['prediction'] < 2, 'prediction'] = 0

header = ['ltable_id', 'rtable_id', 'prediction']
#send output to file
final_merge.to_csv('output.csv',index = False,  columns = header)

ltable_id                  int64
rtable_id                  int64
modelno                    int64
category                 float64
title                    float64
prediction               float64
Product Information_L     object
Product Information_R     object
dtype: object


In [9]:
final_merge

,ltable_id,rtable_id,modelno,category,title,prediction,Product Information_L,Product Information_R
0,11,20440,0,0.0,1.0,0.0,draper high contrast grey targa electric scree...,i.sound 4x foldable portable speaker with batt...
1,12,20440,0,0.0,1.0,0.0,draper at1200 access multiview series e acoust...,i.sound 4x foldable portable speaker with batt...
2,45,12911,0,0.0,1.0,0.0,draper flexible matte white cinefold replaceme...,i.sound 4x foldable portable speaker with batt...
3,45,20440,0,0.0,1.0,0.0,draper flexible matte white cinefold replaceme...,i.sound 4x foldable portable speaker with batt...
4,77,12911,0,0.0,1.0,0.0,draper matte white ultimate access series e el...,i.sound 4x foldable portable speaker with batt...
...,...,...,...,...,...,...,...,...
81862,2418,11689,0,0.0,1.0,0.0,powermat wireless charging system for iphone 3...,powermat three position mat for wireless devic...
81863,2486,4637,1,0.0,0.0,0.0,ofm cafe 41.5 x 30 round folding table_furnitu...,officemate 97228 - plastic coated paper clips ...
81864,2486,17731,1,0.0,0.0,0.0,ofm cafe 41.5 x 30 round folding table_furnitu...,new-officemate 83356 - recycled plastic forms ...
81865,2486,17732,1,0.0,0.0,0.0,ofm cafe 41.5 x 30 round folding table_furnitu...,new-officemate 83301 - portable storage clipbo...


# Part 3
Report the false positives and false negatives by comparing your matches with the ground truth (which is in the testing dataset test.csv we provided). You should also report the precision and recall of your solution and the F1 measure.

In [10]:
#get the ground truth
test = pd.read_csv('test.csv', encoding='latin-1')
test.rename(columns = {'label': 'prediction'}, inplace = True)
test = test[['ltable_id', 'rtable_id', 'prediction']]
#get the output from the Record Linkage performed above
prediction = pd.read_csv('output.csv', encoding = "latin-1")
prediction.reset_index(drop = True)
test.reset_index(drop = True)
intersection = prediction.merge(test, left_on = ['ltable_id', 'rtable_id'], right_on = ['ltable_id', 'rtable_id'])

intersection['Val_Diff'] = intersection['prediction_x'] - intersection['prediction_y']

#calculate confusion matrix boxes
false_positive = intersection[intersection['Val_Diff'] == 1.0].count()['Val_Diff']
true_positive = intersection[intersection['Val_Diff'] == 0.0].count()['Val_Diff']
false_negative = intersection[intersection['Val_Diff'] == -1.0].count()['Val_Diff']
print(false_positive)
print(true_positive)
print(false_negative)

#calculate measures of performance
recall = true_positive / (true_positive + false_negative)
precision = true_positive / (true_positive + false_positive)
F_1 = 2 * (precision * recall) / (precision + recall)
print(recall)
print(precision)
print(F_1)

0
4888
587
0.8927853881278539
1.0
0.9433561709929558


# Part 4
4. Describe in a markdown cell in your notebook the process you used for your solution. Why did you choose the specific index, why the specific attributes (if any) for your blocked index, why did you use or not the SortedNeighborhood algorithm for your index. We want to understand the process you used and we expect it to be  a bit more sophisticated  than  a random pick of parameters. In other words - defend your notebook! 

I thought that entities were unlikely to be the same if the brands didn't match. Therefore, I wanted to index on it. A blocked index only allows for exact matches, so I chose SortedNeighborhood to add some flexibility for spelling errors.
Model numbers are specific to products, and products could have model numbers that are similar. So, that attribute would need to be an exact match in order to be the same entity. The title and category are also good attributes to use for comparisons with moderate thresholds. The category relatively short, so the levenshtein method of counting necessary edits had a higher F-1 score and was used. The title can be long, so it uses the jarowinkler method because that is based on the common characters between the entities and had a higher F-1 score. There was blocking on one attribute and comparisons using three other attributes. The price attribute was not useful for this as it varies a lot between Walmart and Amazon.

For merging the results to a csv, the prediction was set to 1 for a match when 2 or more attributes matched. The prediction value was 0 for not a match when zero or one attributes matched, because these could be due to something like the same category which is not enough to say they are the same entity.
This process resulted in a high recall, precision, and F-1 scores.